# Local settings

In [ ]:
data_dir = 'raw_data/kana_colourpreferencequalia-master/data'

# Specify the data column to focus on throughout by default
data_column = 'responseTime'

# Specify which type of responses to filter for
response_type = 'similarity'

# Import Packages

In [ ]:
import os
import pickle as pkl
import sys
sys.path.append('/Users/kana/Library/Mobile Documents/com~apple~CloudDocs/Codes/GWOT_colorprefrencequalia')
import numpy as np
import pandas as pd
import sklearn
import csv
from scipy.stats import pearsonr, spearmanr
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from mpl_toolkits.mplot3d import Axes3D
from sklearn.manifold import MDS
import seaborn as sns
import ot
import plotly.graph_objs as go
import plotly.express as px
from mpl_toolkits.axes_grid1 import make_axes_locatable

# Our own utility functions
import utilityFunctions

# Define variables/functions (used across the code)

In [ ]:
# Define unique colors
# TODO - remove after confirming that it is no longer needed (replaced by function)
unique_colours = np.array(['#d2b700', '#db8b08', '#c7512c', '#c13547', '#a03663', '#753a7a', '#4b488e', '#005692', '#006a8b', '#007b75', '#008a52', '#9aa400'])
colour_index = {colour: idx for idx, colour in enumerate(unique_colours)}
matrix_size = len(unique_colours)

# Provides dictionary of colours, with an assigned id for each colour
# OUTPUTS:
#   colour_index: dictionary, {colour_string : colour_id}
def getUniqueColours():
    unique_colours = np.array(['#d2b700', '#db8b08', '#c7512c', '#c13547', '#a03663', '#753a7a', '#4b488e', '#005692', '#006a8b', '#007b75', '#008a52', '#9aa400'])
    colour_index = {colour: idx for idx, colour in enumerate(unique_colours)}
    return colour_index

### configuration
n_eps = 15 # number of epsilon values tried
eps_range = [0.04, 5] # the range of epsilon searched
epsilons = np.logspace(np.log10(eps_range[0]), np.log10(eps_range[1]), n_eps) # epsilon values

# Function(s) for loading and formatting data

In [ ]:
# Function for getting data

# Extract data from datafiles in provided directory
#   Data is extracted in the colour x colour matrix form
# INPUTS:
#   data_dir: string, directory holding csv data files
#   target_data: string, target column name in data files which has the data you want to get
#   response_type: string, select data for 'similarity' or 'preference'
# OUTPUTS:
#   pMatrices:
#   pIds:
#   pFiles:
# TODO add parameter which allows for filtering of data based on specified column-value pairs
def getDataColourMatrix(data_dir, target_data, response_type):

    pFiles = [] # stores source datafile for each participant, pFiles[pID] gives the file for participant pID
    pMatrices = [] # stores data matrix for each participant

    pCounter = 0 
    for filename in sorted(os.listdir(data_dir), reverse=False):
        if filename.endswith(".csv"):
            filepath = os.path.join('raw_data/kana_colourpreferencequalia-master/data', filename)

            # This is another participant
            pFiles.append(filename)
            pCounter = pCounter + 1
            
            # Load the CSV file
            df = pd.read_csv(filepath)

            # Filter for rows where 'response_type' is 'similarity' or 'preference'
            df_similarity = df[df['response_type'] == response_type]

            # Extract columns
            colour1 = df_similarity['colour1']
            colour2 = df_similarity['colour2']
            target_preference = df_similarity[target_data]

            # Create and fill the matrix
            # NOTE double pass trials will overwrite the first trials (is this what we want?)
            colour_index = getUniqueColours()
            matrix_size = len(colour_index)
            matrix = np.zeros((matrix_size, matrix_size))
            for c1, c2, tp in zip(colour1, colour2, target_preference):
                I = colour_index[c1]
                j = colour_index[c2]
                matrix[j, I] = tp

            # Store the matrix
            pMatrices.append(matrix)

    pIds = range(0, pCounter)

    return pMatrices, pIds, pFiles

# Show data

In [ ]:
# Get data
pMatrices, pIds, pFiles = getDataColourMatrix(data_dir, data_column, response_type)

if data_column == 'responseTime':
    cLabel = 'response time (s)'
else:
    cLabel = response_type

# Show matrices
utilityFunctions.show_heatmaps(0, 7, matrices=pMatrices, titles=["subject" + str(p) for p in pIds], cbar_label= cLabel, color_labels=unique_colours)

for f in range(0, len(pFiles)):
    print("subject" + str(f) + ' ' + pFiles[f])


In [ ]:
pCorrs = []

folder_path = 'raw_data/kana_colourpreferencequalia-master/data'

for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        filepath = os.path.join(folder_path, filename)
        
        df = pd.read_csv(filepath, usecols=['response_type', 'response', 'colour1', 'colour2', 'trials.thisIndex', 'practice_trial'])

        # Omit practice trials
        df = df[df['practice_trial'] != 1]  

        df_similarity = df[df['response_type'] == 'similarity']

        # Group by 'trials.thisIndex'
        grouped = df_similarity.groupby('trials.thisIndex')['response'].apply(list).reset_index()

        # Filter for pairs with exactly two trials
        repeated_pairs = grouped[grouped['response'].apply(len) == 2]

        # Check if there are any repeated pairs
        if len(repeated_pairs) > 0:
            # Flatten the responses for repeated pairs
            responses_trial1 = np.array(repeated_pairs['response'].apply(lambda x: x[0]).tolist())
            responses_trial2 = np.array(repeated_pairs['response'].apply(lambda x: x[1]).tolist())

            # Calculate overall correlation for repeated color pairs
            overall_correlation = pearsonr(responses_trial1, responses_trial2)[0]

            pCorrs.append(overall_correlation)

            print(f"Overall Pearson Correlation for {filename}: {overall_correlation}")
        else:
            print(f"No repeated pairs found in {filename}")
    
    plt.hist(pCorrs, color='blue')
    plt.xlabel("r")
    plt.ylabel("N")
    plt.title('double pass - similarity')